In [394]:
import numpy as np
import nltk
from collections import defaultdict
from collections import Counter
from src.classes.database.offer import MongoOffer
from tqdm import tqdm
from enum import Enum
import re
import json

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [395]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
tagged = nltk.pos_tag(token_list[0])

direction = Enum('direction', ['FORWARD', 'BACKWARD', 'SELF'])

In [396]:
def get_all_words(token_list: list[list]) -> list:
    words = []
    for sentence in token_list:
        for word in sentence:
            words.append(word)
    return sorted(set(words))

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

# Old method, slow and not used anymore
def calculate_all_probabilities(tokens: list[list], look_ahead) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = defaultdict()
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [defaultdict(float) for x in range(0,look_ahead)]
        for x in range(0,look_ahead):
            for sub_word in all_words:
                p = p_next_word(word, sub_word, x+1, tokens)
                if p > 0:
                    p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")
    return p_word
#data = calculate_all_probabilities(token_list, 1)

## HMaxtrix
Used for storing all word occurrences in a 3 dimensional matrix

### Data we can get from matrix
 1. p(start, find) = probability that 'find' is a n-order forward word for 'start' | single cell value / sum of 'start' row
 2. p(start, find) = probability that 'find' is a n-order backward word for 'start' | single cell value / sum of 'start' column
 3. p(row_word) = probability that 'row word' will be a n-order forward word | sum of row / entire table sum
 4. p(col_word) = probability that 'col word' will be a n-order backward word | sum of column / entire table sum
 5. p(word, order) = probability that 'word' will be n-order word compared to the other orders| single cell value at order / all order values of that word tallied (through the table)
 6.  p(word-row, order) = probability that 'word' will be n-order forward word | all of word-row order sum / all order values of that word-row
 7. p(col-row, order) = probability that 'word' will be n-order backward word | all of word-col order sum / all order values of that word-col

In [397]:
class HMatrix:
    def __init__(self) -> None:
        self.labels = None
        self.reverse_labels = None
        self.order = 0
        self.matrix = None

    def p_row_word(self, order: int, row_word: str, word: str) -> float:
        if row_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        row_label = self.reverse_labels[row_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, row_label, :][word_label]
        divisor = sum(self.matrix[order, row_label, :])

        return dividend/divisor if divisor > 0 else 0

    def p_col_word(self, order: int, col_word: str, word: str) -> float:
        if col_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        col_label = self.reverse_labels[col_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, :, col_label][word_label]
        divisor = sum(self.matrix[order, :, col_label])

        return dividend/divisor if divisor > 0 else 0


    def create_matrix(self, tokens: list[list], order: int):
        #Setup
        self.labels = get_all_words(token_list)
        self.reverse_labels = {self.labels[x]: x for x in range(0, len(self.labels))}
        self.order = order
        self.matrix = np.zeros((order, len(self.labels), len(self.labels)))

        #Iteration
        for sentence in tqdm(tokens):
            for x in range(0, len(sentence)):
                word = sentence[x]
                print(f"Word: {word} ({str(self.reverse_labels[word])})")
                for y in range(0, self.order):
                    if x+y+1 < len(sentence):
                        print(f"Lookahead {str(y+1)}: {sentence[x+y+1]} ({str(self.reverse_labels[sentence[x+y+1]])})")
                        self.matrix[y][self.reverse_labels[word]][self.reverse_labels[sentence[x+y+1]]] += 1

In [416]:
matrix = HMatrix()
matrix.create_matrix(token_list, 8)

100%|██████████| 2821/2821 [00:00<00:00, 12972.14it/s]

Word: logitech (1926)
Lookahead 1: g (1492)
Lookahead 2: pro (2372)
Lookahead 3: x (3162)
Lookahead 4: superlight (2827)
Word: g (1492)
Lookahead 1: pro (2372)
Lookahead 2: x (3162)
Lookahead 3: superlight (2827)
Word: pro (2372)
Lookahead 1: x (3162)
Lookahead 2: superlight (2827)
Word: x (3162)
Lookahead 1: superlight (2827)
Word: superlight (2827)
Word: nintendo (2138)
Lookahead 1: switch (2844)
Lookahead 2: lite (1914)
Lookahead 3: gray (1604)
Lookahead 4: with (3146)
Lookahead 5: original (2218)
Lookahead 6: box (1013)
Lookahead 7: extras (1396)
Lookahead 8: anbernic (845)
Word: switch (2844)
Lookahead 1: lite (1914)
Lookahead 2: gray (1604)
Lookahead 3: with (3146)
Lookahead 4: original (2218)
Lookahead 5: box (1013)
Lookahead 6: extras (1396)
Lookahead 7: anbernic (845)
Lookahead 8: rg351p (2509)
Word: lite (1914)
Lookahead 1: gray (1604)
Lookahead 2: with (3146)
Lookahead 3: original (2218)
Lookahead 4: box (1013)
Lookahead 5: extras (1396)
Lookahead 6: anbernic (845)
Lookahead

## Sentence Class

Input a sentence, it will then use the HMatrix to see which path results the most likely outcome for products

In [418]:
class Sentence:
    def __init__(self, matrix: HMatrix, tokenizer, raw_sentence: str, verbose=False):
        self.verbose = verbose
        self.raw_sentence = tokenizer.tokenize(raw_sentence.lower())
        self.sentence = []

        # Creating all the blank word classes
        for x, word in enumerate(self.raw_sentence):
            self.sentence.append(Word(word, x))

        # Creating all the forward connections
        for x, word in enumerate(self.raw_sentence):
            print(f"--- {word} ---")
            for neighbor in self.sentence:
                mapped_pos = self.map_word_pos_to_order(neighbor.position-x)

                if mapped_pos[1] is not direction.SELF and mapped_pos[0] < matrix.order: #Looking at itself and order is within scope

                    if mapped_pos[1] is direction.FORWARD and self.verbose:
                        self.sentence[x].neighbors.append(
                            {'word': neighbor.key,
                             'p': matrix.p_row_word(mapped_pos[0], word, neighbor.key),
                             'direction': mapped_pos[1].name
                            })
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"-{str(mapped_pos[0])}-> "
                              f"{neighbor.key} "
                              f"({str(matrix.p_row_word(mapped_pos[0], word, neighbor.key))})")
                    else:
                        self.sentence[x].neighbors.append(
                            {'word': neighbor.key,
                             'p': matrix.p_col_word(mapped_pos[0], word, neighbor.key),
                             'direction': mapped_pos[1].name
                            })
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"<-{str(mapped_pos[0])}- "
                              f"{neighbor.key} "
                              f"({str(matrix.p_col_word(mapped_pos[0], word, neighbor.key))})")

                # if word == neighbor.key and x == neighbor.position:
                #     word.forward_neighbors.append({'word': neighbor.key, 'p_forward': -1})
                # else:
                #     word.forward_neighbors.append({'word': neighbor.key, 'p_forward': matrix.p_row_word(neighbor.pos, word, neighbor.key)})
                #
    def map_word_pos_to_order(self, position) -> tuple:
        #Forward word positions
        if position > 0:
            return position-1, direction.FORWARD

        # Backwards word positions
        if position < 0:
            return abs(position)-1, direction.BACKWARD

        #Pos looking at self
        if position == 0:
            return -1, direction.SELF



class Word:
    def __init__(self, key: str, position: int):
        self.key = key
        self.position = position
        self.neighbors = []

    def __str__(self):
        return f"Word: {self.key} | Position: {str(self.position)}"


In [421]:
test = Sentence(matrix, tokenizer, "logitech g pro x superlight intel i7 8700k", verbose=True)

--- logitech ---
logitech -0-> g (0.24390243902439024)
  logitech -1-> pro (0.25)
    logitech -2-> x (0.15789473684210525)
      logitech -3-> superlight (0.11538461538461539)
        logitech -4-> intel (0.0)
          logitech -5-> i7 (0.0)
            logitech -6-> 8700k (0.0)
--- g ---
g <-0- logitech (0.136986301369863)
g -0-> pro (0.11764705882352941)
  g -1-> x (0.08823529411764706)
    g -2-> superlight (0.047619047619047616)
      g -3-> intel (0.0)
        g -4-> i7 (0.0)
          g -5-> 8700k (0.0)
--- pro ---
  pro <-1- logitech (0.042735042735042736)
pro <-0- g (0.03460207612456748)
pro -0-> x (0.025454545454545455)
  pro -1-> superlight (0.012)
    pro -2-> intel (0.0045045045045045045)
      pro -3-> i7 (0.0)
        pro -4-> 8700k (0.0)
--- x ---
    x <-2- logitech (0.08571428571428572)
  x <-1- g (0.075)
x <-0- pro (0.07777777777777778)
x -0-> superlight (0.0375)
  x -1-> intel (0.0)
    x -2-> i7 (0.0)
      x -3-> 8700k (0.0)
--- superlight ---
      superlight <-

In [420]:
test.sentence[0].neighbors

[{'word': 'g', 'p': 0.24390243902439024, 'direction': 'FORWARD'},
 {'word': 'pro', 'p': 0.25, 'direction': 'FORWARD'},
 {'word': 'x', 'p': 0.15789473684210525, 'direction': 'FORWARD'},
 {'word': 'superlight', 'p': 0.11538461538461539, 'direction': 'FORWARD'},
 {'word': 'intel', 'p': 0.0, 'direction': 'FORWARD'},
 {'word': 'i7', 'p': 0.0, 'direction': 'FORWARD'},
 {'word': '8700k', 'p': 0.0, 'direction': 'FORWARD'}]

In [21]:
test.matrix[0, :, 423][1994]

1.0